<a href="https://colab.research.google.com/github/sulaimanbehzad/Clustering_according_to_item_popularity/blob/main/price_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estimating the price of each product
We will use Neural Network to predict the price of each product

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, normalize, MinMaxScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso 
from sklearn.metrics import explained_variance_score as evs
from sklearn.metrics import r2_score as r2
from sklearn import metrics

In [ ]:
transactions = pd.read_csv('tarikhche kharid.csv')
orders = pd.read_csv('orders.csv')
transactions.head()

,id,product_variant_id,selling_price,rrp_price,base_price,buy_price,order_limit,start_at,end_at,tags,show_in_price_history,active,created_at,product_id,marketplace_seller_id
0,18752043,1977385,850000,850000,\N,\N,1,\N,\N,\N,1,1,2018-11-26 18:59:52,667969,2962
1,18752022,1977380,12550000,12550000,\N,\N,2,\N,\N,\N,1,0,2018-11-26 18:59:31,150144,78525
2,18752046,1977380,12500000,12500000,\N,\N,2,\N,\N,\N,1,1,2018-11-26 18:59:55,150144,78525
3,18751982,1977374,310000,310000,\N,\N,2,\N,\N,\N,1,0,2018-11-26 18:58:48,908541,4199
4,18752018,1977374,300000,300000,\N,\N,2,\N,\N,\N,1,1,2018-11-26 18:59:28,908541,4199


In [ ]:
orders.head()

,ID_Order,ID_Customer,ID_Item,DateTime_CartFinalize,Amount_Gross_Order,city_name_fa,Quantity_item
0,2714054,469662,21386,2015-10-15 08:50:56.000,597982.0,محمود آباد,1.0
1,11104039,3063877,248497,2018-02-11 00:29:26.000,980000.0,خرمدره,1.0
2,4228130,3184893,50144,2016-06-14 00:30:08.000,229358.0,قرچک,1.0
3,22225624,6888562,70208,2018-09-03 14:37:19.000,16514.0,قم,1.0
4,4068771,2533490,67627,2016-05-21 11:51:02.000,133028.0,تهران,1.0


### Feature Selection


In [ ]:
orders = orders.filter(['ID_Item', 'Amount_Gross_Order', 'Quantity_item'])

In [ ]:
orders['Unit_Price'] = np.divide(orders['Amount_Gross_Order'], orders['Quantity_item'])
orders['Unit_Price']

0         597982.0
1         980000.0
2         229358.0
3          16514.0
4         133028.0
            ...   
199995    172018.5
199996    600000.0
199997    450000.0
199998    193486.0
199999     55046.0
Name: Unit_Price, Length: 200000, dtype: float64

In [ ]:
orders.head(30)

,ID_Item,Amount_Gross_Order,Quantity_item,Unit_Price
0,21386,597982.0,1.0,5.979820e+05
1,248497,980000.0,1.0,9.800000e+05
2,50144,229358.0,1.0,2.293580e+05
3,70208,16514.0,1.0,1.651400e+04
4,67627,133028.0,1.0,1.330280e+05
5,312932,290000.0,1.0,2.900000e+05
6,42365,55046.0,1.0,5.504600e+04
7,715680,2790000.0,1.0,2.790000e+06
8,102588,34862.0,1.0,3.486200e+04
9,658519,2600000.0,1.0,2.600000e+06


Let's inspect the database

In [ ]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ID_Item             200000 non-null  int64  
 1   Amount_Gross_Order  200000 non-null  float64
 2   Quantity_item       200000 non-null  float64
 3   Unit_Price          200000 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 6.1 MB


In [ ]:
orders.isnull().sum()

ID_Item               0
Amount_Gross_Order    0
Quantity_item         0
Unit_Price            0
dtype: int64

In [ ]:
transactions = transactions.filter(['id', 'product_variant_id', 'selling_price', 'product_id', 'marketplace_seller_id'])
transactions.head(10)

,id,product_variant_id,selling_price,product_id,marketplace_seller_id
0,18752043,1977385,850000,667969,2962
1,18752022,1977380,12550000,150144,78525
2,18752046,1977380,12500000,150144,78525
3,18751982,1977374,310000,908541,4199
4,18752018,1977374,300000,908541,4199
5,18751959,1977367,15500000,150144,78525
6,18752064,1977367,12500000,150144,78525
7,18751958,1977366,750000,197348,11436
8,18751880,1977358,15800000,150144,78525
9,18751863,1977354,3500000,485078,66785


### Translation for Products

In [ ]:
# !pip install google_trans_new

In [ ]:
# from google_trans_new import google_translator

In [ ]:
# translator = google_translator()

In [ ]:
# translated_title = []
# for title in products['product_title_fa']:
#     translated_title.append(translator.translate(title, lang_src='fa', lang_tgt='english'))

In [ ]:
# translated_title_alt = []
# for text in products['title_alt']:
#     translated_title_alt.append(translator.translate(text, lang_src='fa', lang_tgt='english'))

In [ ]:
# translated_category = []

In [ ]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2184358 entries, 0 to 2184357
Data columns (total 5 columns):
 #   Column                 Dtype
---  ------                 -----
 0   id                     int64
 1   product_variant_id     int64
 2   selling_price          int64
 3   product_id             int64
 4   marketplace_seller_id  int64
dtypes: int64(5)
memory usage: 83.3 MB


In [ ]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ID_Item             200000 non-null  int64  
 1   Amount_Gross_Order  200000 non-null  float64
 2   Quantity_item       200000 non-null  float64
 3   Unit_Price          200000 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 6.1 MB


In [ ]:
df = pd.merge(transactions, orders, left_on='product_variant_id', right_on='ID_Item')
df.head()

,id,product_variant_id,selling_price,product_id,marketplace_seller_id,ID_Item,Amount_Gross_Order,Quantity_item,Unit_Price
0,18751958,1977366,750000,197348,11436,1977366,670000.0,1.0,670000.0
1,18750189,1977141,360000,140514,83712,1977141,250000.0,1.0,250000.0
2,18748963,1976967,60000,498137,83712,1976967,55000.0,4.0,13750.0
3,18741603,1975894,195000,125726,856,1975894,195000.0,1.0,195000.0
4,18750097,1975894,195000,125726,856,1975894,195000.0,1.0,195000.0


In [ ]:
df.drop_duplicates(subset=['product_id'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176129 entries, 0 to 176128
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     176129 non-null  int64  
 1   product_variant_id     176129 non-null  int64  
 2   selling_price          176129 non-null  int64  
 3   product_id             176129 non-null  int64  
 4   marketplace_seller_id  176129 non-null  int64  
 5   ID_Item                176129 non-null  int64  
 6   Amount_Gross_Order     176129 non-null  float64
 7   Quantity_item          176129 non-null  float64
 8   Unit_Price             176129 non-null  float64
dtypes: float64(3), int64(6)
memory usage: 13.4 MB


In [ ]:
df.head(30)

,id,product_variant_id,selling_price,product_id,marketplace_seller_id,ID_Item,Amount_Gross_Order,Quantity_item,Unit_Price
0,18751958,1977366,750000,197348,11436,1977366,670000.0,1.0,670000.0
1,18750189,1977141,360000,140514,83712,1977141,250000.0,1.0,250000.0
2,18748963,1976967,60000,498137,83712,1976967,55000.0,4.0,13750.0
3,18741603,1975894,195000,125726,856,1975894,195000.0,1.0,195000.0
4,18750097,1975894,195000,125726,856,1975894,195000.0,1.0,195000.0
5,18730504,1974569,525000,469247,70532,1974569,495000.0,1.0,495000.0
6,18724640,1973834,475000,147614,7715,1973834,430000.0,1.0,430000.0
7,18724722,1973834,475000,147614,7715,1973834,430000.0,1.0,430000.0
8,18724856,1973834,475000,147614,7715,1973834,430000.0,1.0,430000.0
9,18724878,1973834,470000,147614,7715,1973834,430000.0,1.0,430000.0


In [ ]:
prices = df['selling_price']
prices

0         750000
1         360000
2          60000
3         195000
4         195000
           ...  
176124    124400
176125    119800
176126    118800
176127    118400
176128    117800
Name: selling_price, Length: 176129, dtype: int64

<hr>

In [ ]:
temp = df.drop(columns=['Unit_Price'])
X = temp.values
X

array([[1.8751958e+07, 1.9773660e+06, 7.5000000e+05, ..., 1.9773660e+06,
        6.7000000e+05, 1.0000000e+00],
       [1.8750189e+07, 1.9771410e+06, 3.6000000e+05, ..., 1.9771410e+06,
        2.5000000e+05, 1.0000000e+00],
       [1.8748963e+07, 1.9769670e+06, 6.0000000e+04, ..., 1.9769670e+06,
        5.5000000e+04, 4.0000000e+00],
       ...,
       [1.8280403e+07, 1.1550660e+06, 1.1880000e+05, ..., 1.1550660e+06,
        1.3110000e+05, 3.0000000e+00],
       [1.8287245e+07, 1.1550660e+06, 1.1840000e+05, ..., 1.1550660e+06,
        1.3110000e+05, 3.0000000e+00],
       [1.8416565e+07, 1.1550660e+06, 1.1780000e+05, ..., 1.1550660e+06,
        1.3110000e+05, 3.0000000e+00]])

In [ ]:
Y = prices.values
Y

array([750000, 360000,  60000, ..., 118800, 118400, 117800], dtype=int64)

In [ ]:
Y.shape

(176129,)

In [ ]:
X.shape

(176129, 8)

In [ ]:
min_max_scaler = MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
X_scale

array([[9.99991828e-01, 1.00000000e+00, 5.99998624e-12, ...,
        1.00000000e+00, 7.38181601e-03, 0.00000000e+00],
       [9.99841251e-01, 9.99726377e-01, 2.87998924e-12, ...,
        9.99726377e-01, 2.64916035e-03, 0.00000000e+00],
       [9.99736894e-01, 9.99514776e-01, 4.79991539e-13, ...,
        9.99514776e-01, 4.51855933e-04, 1.03448276e-01],
       ...,
       [9.59853107e-01, 0.00000000e+00, 9.50391088e-13, ...,
        0.00000000e+00, 1.30936807e-03, 6.89655172e-02],
       [9.60435497e-01, 0.00000000e+00, 9.47191091e-13, ...,
        0.00000000e+00, 1.30936807e-03, 6.89655172e-02],
       [9.71443205e-01, 0.00000000e+00, 9.42391095e-13, ...,
        0.00000000e+00, 1.30936807e-03, 6.89655172e-02]])

In [ ]:
X_scale.shape

(176129, 8)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X_scale,Y,test_size=0.2,random_state=123)

In [ ]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(140903, 8) (35226, 8) (140903,) (35226,)


In [ ]:
# 1. OLS

ols = LinearRegression(normalize=True)
ols.fit(x_train, y_train)
ols_yhat = ols.predict(x_test)

# 2. Lasso

lasso = Lasso(alpha = 10)
lasso.fit(x_train, y_train)
lasso_yhat = lasso.predict(x_test)

C:\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.2018521116334554e+22, tolerance: 5.445645303303538e+20
  model = cd_fast.enet_coordinate_descent(


In [ ]:
print(('EXPLAINED VARIANCE SCORE:'))
print('-------------------------------------------------------------------------------')
print(('Explained Variance Score of OLS model is {}'.format(evs(y_test, ols_yhat).round())))
print(('Explained Variance Score of Lasso model is {}'.format(evs(y_test, lasso_yhat).round())))


EXPLAINED VARIANCE SCORE:
-------------------------------------------------------------------------------
Explained Variance Score of OLS model is 1.0
Explained Variance Score of Lasso model is 1.0


In [ ]:
print('R-SQUARED:')
print('-------------------------------------------------------------------------------')
print(('R-Squared of OLS model is {}'.format(r2(y_test, ols_yhat).round())))
print(('R-Squared of Lasso model is {}'.format(r2(y_test, lasso_yhat).round())))


R-SQUARED:
-------------------------------------------------------------------------------
R-Squared of OLS model is 1.0
R-Squared of Lasso model is 1.0
